## Import Dependencies

In [1]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import LLMChain
from langchain.tools import DuckDuckGoSearchRun
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

#Playwright
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,
    create_sync_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.
)

#Python Tools
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply()

#Files ToolKit
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.agents.agent_toolkits import FileManagementToolkit
from tempfile import TemporaryDirectory

# We'll make a temporary directory to avoid clutter
working_directory = TemporaryDirectory()

## Load LLM

In [18]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
model_path = "../../../../../Models/GGML/llama-2-7b-chat.ggmlv3.q8_0.bin"
n_batch = 512 
llm = LlamaCpp(
    model_path=model_path,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

llama.cpp: loading model from ../../../../../Models/GGML/llama-2-7b-chat.ggmlv3.q8_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 7 (mostly Q8_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 6828.73 MB (+ 1024.00 MB per state)
llama_new_context_with_model: kv self size 

## Load Tools

In [19]:
async_browser = create_async_playwright_browser()
scrape_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
scraping_tools = scrape_toolkit.get_tools()

In [20]:
tools_by_name = {tool.name: tool for tool in scraping_tools}
print(tools_by_name.keys())
navigate_tool = tools_by_name["navigate_browser"]
extract_text = tools_by_name["extract_text"]

dict_keys(['click_element', 'navigate_browser', 'previous_webpage', 'extract_text', 'extract_hyperlinks', 'get_elements', 'current_webpage'])


In [21]:
files_tools = FileManagementToolkit(
    root_dir=str(working_directory.name),
    selected_tools=["read_file", "write_file", "list_directory"],
).get_tools()
read_tool, write_tool, list_tool = files_tools

In [22]:
DuckDuckGoSearch = DuckDuckGoSearchRun()
PythonTool =PythonREPLTool()

tools = [
    Tool(
        name="DuckDuckGo",
        func=DuckDuckGoSearch.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Read directory",
        func=read_tool.run,
        description="Read files within a directory",
    ),
    Tool(
        name="Write directory",
        func=write_tool.run,
        description="Write files to a directory",
    ),
    Tool(
        name="Extract text from webpage",
        func=extract_text.run,
        description="Extracts text from a webpage",
    ),
    extract_text,
    navigate_tool, PythonTool, 
]

In [24]:
prefix = """Answer the following questions as best you You have access to the following tools:"""
suffix = """Begin! Remember to be as creative as you can"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)

In [25]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [26]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [27]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [28]:
agent_executor.run("""
Help me write an API service based on the use case below. I want this to host this on my own computer for now before deploying it onto the cloud. Please provide a guide as to how I can accomplish this. Break this down into phases so that we dont go over your context window. Just list the steps for now. i will then ask you about each.  When you are asked for code. be as concise as possible. Try not to miss out on anything important or skip any steps as I will be following you step by step. 

Remember, give me the high level overview for now. I will ask you for the implementation later.

API Use Case: a web scraper that scrapes a site's details
Details ton extract: Tender Submission Date, Tender Derpartment, Tender Description, Tender Closing Date, Tender Contact person.
API Service Programming Language/Stack/Technologies: Python
API Architecture: Microservices


Notes:
The tenders are listed in paginated views. The details page is on a different page and contains details like the Tender Contact Person and the files assocaited with the tender

Example listing:
https://easytenders.co.za/tenders?sort=&search=IT&province=gauteng&company=&industry=any&status=open-tenders&filter=1

Example Details Page: https://easytenders.co.za/tenders/request-for-bidopen-tender-ctt-2023-2024-002-website-ctt-2023-2024-002-website-redevelopment-120710""")



> Entering new AgentExecutor chain...
End Notes



llama_print_timings:        load time = 108015.48 ms
llama_print_timings:      sample time =     3.88 ms /     4 runs   (    0.97 ms per token,  1030.13 tokens per second)
llama_print_timings: prompt eval time = 148085.94 ms /   677 tokens (  218.74 ms per token,     4.57 tokens per second)
llama_print_timings:        eval time = 15829.92 ms /     3 runs   ( 5276.64 ms per token,     0.19 tokens per second)
llama_print_timings:       total time = 163965.28 ms


OutputParserException: Could not parse LLM output: `End Notes
`